In [8]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

park = Customer('park', 100)
park

class LineItem:
  """구매할 물품/갯수 생성해서 총 가격을 반환"""
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price*self.quantity

In [9]:
class Order:
  """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion = None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    """__total 속성이 없으면 전체 계산할 값을 계산"""
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
      return self.__total

    def due(self):
      """할인 금액 차감"""
      if self.promotion is None:
        discount = 0
      else:
        discount = self.promotion.discount(self)
      return self.total() - discount

    def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [10]:
class Promotion(ABC):
  """할인 혜택 클래스들의 형태를 선언"""

  @abstractmethod
  def discount(self, order):
    """할인액을 구체적인 숫자로 반환"""
    pass

In [11]:
class FidelityPromo(Promotion):
  """충성도 점수가 1000점 이상인 고객에게 전체 5%할인 적용"""

  def discount(self, order):
    return order.total() * 0.005 if order.customer.fidelity>= 1000 else 0

class BulkItemPromo(Promotion):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [12]:
joe = Customer('John Doe', 0 )
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

Order(joe, cart, FidelityPromo())
Order(ann, cart, FidelityPromo())



In [13]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, BulkItemPromo())

long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

Order(joe, long_order, LargeOrderPromo())
Order(joe, cart, LargeOrderPromo())

In [14]:
class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산한 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


In [15]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


In [17]:
Order(joe, cart, fidelity_promo)

Order(ann, cart, fidelity_promo)

Order(joe, banana_cart, bulk_item_promo)

Order(ann, long_order, large_order_promo)


<Order total: 10.00 due: 9.30>

In [18]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

Order(joe, long_order, best_promo)

Order(joe, banana_cart, best_promo)

Order(ann, cart, best_promo)


<Order total: 42.00 due: 39.90>

In [19]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

promos


[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]